# Review : Fast R-CNN

>* edwith '딥러닝의 맥' 강의 내용과 함께 SH Tsang의 [Review: Fast R-CNN](https://medium.com/coinmonks/review-fast-r-cnn-object-detection-a82e172e87ba)을 번역한 글입니다.
>
>* 논문 : [2015 ICCV, Fast R-CNN](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Girshick_Fast_R-CNN_ICCV_2015_paper.pdf)

## Introduction

이번 글에서는 Fast R-CNN(Fast Region-based Convolutional Network method)를 알아보도록 하겠다. 

Classification 문제와는 달리 object detection 문제를 풀기 위해 여러 단계의 복잡한 방법이 필요하다. Region을 찾고, CNN 모델을 돌리고, 정확한 bounding box 위치를 찾기위한 정제과정까지 필요하기 때문이다. Fast R-CNN 이전의 모델들은 이러한 방법을 따로 학습시키는 multi-stage pipeline을 사용했지만, Fast R-CNN은 학습과정을 하나로 묶을 수 있는 single-stage training algorithm을 제안하고 있다. Microsoft Research에서 발표한 __Fast R-CNN__은 기존의 Object detection 모델보다 좋은 성능을 보여주었으며, training과 testing 속도를 향상시킨 모델이다. 

1. VGG-16 구조로 만든 Fast R-CNN의 경우, 기존 R-CNN보다 학습 시 9배, 테스트 시 213배 빠른 속도를 보여줬다.
2. PASCAL VOC 2012에서 더 높은 mAP를 기록했다.
3. SPPNet과 비교했을 때도 학습 시 3배, 테스트 시 10배 빠른 속도를 보여줬으며, 정확도(accuracy)도 높게 측정되었다.

Fast R-CNN은 2015년 ICCV에 논문을 발표하였으며, 약 3000회의 인용수를 보여주고 있다.

## Architecture

모델의 전체적인 구조(architecture)는 다음과 같으며, multi-task loss를 사용해서 end-to-end 방식으로 학습한다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*67iVyCzqapfB5Nyci_zynw.png" width="500">
    <br>
    <i>Fast R-CNN</i>
</div>
<br>

1. Input은 전체이미지와 object proposals set이다(entire image and a set of object proposals).
    - 논문에서는 object(region) proposals 영역을 RoI라고 칭하는 것 같다.<br>
    _'An input image and multiple regions of interest (RoIs) are input into a fully convolutionalnetwork'_
2. 전체 이미지를 ConvNet을 통과시켜 feature map을 얻는다.
3. Feature map에서 proposal에 있는 영역을 가져온다(crop). -> RoI projection
4. 가져온 RoI 영역을 pooling해서 fixed-size feature map을 만든다.
5. 마지막으로 Fixed-size feature map을 Fully connected layer를 통과시키고, 두 개의 Branch로 나눠져 결과값을 얻는다.

모델의 output은 2가지로 구성되어있다. 
* 첫번째는 class를 분류하는 output(K+1 = K class + 의미없는 class)
* 두번째는 bouding box의 위치를 조정(refined)하는 bouding box regressor output(four-tuple (r, c, h,w) that specifies its top-left corner (r, c) and its height and width (h,w))

<br>
<div align="center">
    <img src="https://user-images.githubusercontent.com/30274701/52835628-8ae28800-312a-11e9-84ea-a2be45a11d45.png" width="400">
    <br>
    <i>Fast R-CNN in edwith slide</i>
</div>
<br>

## Index

1. [The Problems of Prior Arts](#a)
2. [ROI Pooling Layer](#b)
3. [Multi-task Loss](#c)
4. [Some Other Ablation Study](#d)
5. [Comparison with State-of-the-art Results](#e)

<a id='a'></a>
## 1. The Problems of Prior Arts

기존 R-CNN(Region-based Convolutional Network method) 모델은 ConvNet으로 object detection에서 뛰어난 성능을 보여주었지만, 몇 가지 단점이 존재했다.

### 1.1. Training is a multi-stage pipeline

R-CNN은 먼저 region proposals이 통과할 ConvNet을 학습시켜야한다. 그 다음, output으로 계산된 ConvNet features로 분류 모델인 SVM을 학습시키고(Softmax로 대체 할 수 있음), 마지막으로 bounding-box regressor를 학습하는 방식이였다. 따라서 R-CNN과 SPPNet은 end-to-end 학습이라고 할 수 없었다. 

 * end-to-end learning : 처음의 데이터를 끝까지 통과시켜 학습하는 방식.

### 1.2 Expensive in Space and Time

The bounding box regressor를 학습시키기 위해 Feature vector를 하드디스크에 저장하는데, 고용량(hundresd of gigabyte)의 데이터가 발생한다.

### 1.3 Slow Object Detection

테스트 시, GPU 환경에서 R-CNN을 사용하면 이미지 한장 당 47초에가 걸린다.

특히 R-CNN은 각각의 region proposals에 대해 ConvNet을 통과시켜야 했기때문에, 시간이 많이걸렸다. 다음으로 등장한 SPPNet 구조는 한번만 ConvNet을 통과하도록 변형했기 때문에 시간을 단축할 수 있었다. 하지만 SPPNet 구조도 multi-stage pipeline으로 구성되어 여전히 feature vector을 디스크에 저장해야했으며, 각 계층이 함께 학습되지 못하는 단점이 있었다.

>_Like R-CNN, training is a multi-stage pipeline that involves extracting features, fine-tuning a network with log loss, training SVMs,
and finally fitting bounding-box regressors. Features are also written to disk. But unlike R-CNN, the fine-tuning algorithm proposed in [11, SPPNet] cannot update the convolutional layers that precede the spatial pyramid pooling._

__Fast R-CNN__은 이러한 문제점들을 해결한 구조이며, 4가지 장점을 있다고 한다.

>1. Higher detection quality (mAP) than R-CNN, SPPnet
>2. Training is single-stage, using a multi-task loss
>3. Training can update all network layers
>4. No disk storage is required for feature caching

<a id='b'></a>
## 2. ROI Pooling Layer

ROI Pooling은 __Region-of-Interest Pooling__의 약자로, pooling 방법을 의미한다. Fast R-CNN은 SPPNet에서 하나의 __pyramid__ 구조를 사용하는 특정한 경우(case)라고 볼 수 있다. 강의에서는 Fast R-CNN과 SPPNet이 결국 한 conv layer를 통과한 후 pooling 과정에 어떠한 스킬을 넣었는가에 따라 달라진다고 말한다. 아래 그림을 참조해보자. 

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*aB4gy6i8Zc3BasYaQGDVtg.png" width="600">
    <br>
    <i>ROI Pooling</i>
</div>
<br>

Region proposal로 나온 $h×w$의 bounding box가 있다고하자. 이 영역을 pooling layer를 통과시켜 $H×W$ 크기의 output를 얻으려고 한다. 그러면 각 pooling 영역은 $h/H × w/W$가 된다.

위의 예시를 보면, $5×7$의 input ROI에서 $2×2$ output을 얻기위해 반올림해서 $2×3$ 이나 $3×3$ 영역을 pooling area로 정한다.

Conventional max pooling layer처럼 각 grid에서 가장 큰 값(max value)를 output으로 정한다.

<a id='c'></a>
## 3. Multi-task Loss

Fast R-CNN은 object의 class와 동시에 bounding box의 위치와 크기까지 학습하는 end-to-end learning 구조(architecture)이므로, 두 가지를 반영하는 multi-task loss를 사용한다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*YzFseoGKhmDrqagVRJ5_qw.png" width="400">
    <br>
    <i>Multi-task Loss</i>
</div>
<br>

* $L_cls$는 class에 관련된 loss값이다.
* $L_los$는 bounding box에 관련된 loss값이다.
* $[u≥1]$는  $u≥1$이면 1로 만들어주겠다는 의미이다(u=0 is background class).

아래 표는 Multi-task loss가 효과가 있는지 실험한 결과이다. 실험은  bounding box regressor 옵션을 추가하면서 기존 loss와 multi-task loss를 비교했다.

* S = AlexNet or CaffeNet
* M = VGG-like wider version of S
* L = VGG-16

* State-wise training : softmax 부분과 bounding box regressor를 따로 분리해서 학습하는 방식

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*i0Fq3zitbotf8mZBURqe7w.png" width="600">
    <br>
    <i>Multi-task Loss Results</i>
</div>
<br>

첫번째 Column은 기존 loss, 두번째 Column은 multi-task loss를 사용한 방법으로 테스트한 결과이다. S,M,L 모든 모델에서 multi-task loss를 사용하면 성능이 향상된 점을 볼 수 있으며, Bounding box regressor를 추가한 모델에서도 multi-task loss 방식이 더 좋은 성능을 보여주고 있다.

<a id='d'></a>
## 4. Some Other Ablation Study

### 4.1 Multi Scale Training and Testing

5가지 scale의 Input image를 사용해서 테스트해보았다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*awn21lCgHv2hKImdlfXSpg.png" width="400">
    <br>
    <i>1-Scale vs 5-Scale</i>
</div>
<br>

5 scale을 사용하면 더 많은 test rate(seconds/image) 비용이 발생했지만, 높은 mAP를 얻을 수 있었다.

### 4.2 SVM vs Softmax

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*5VLipfjGezKosdjTADi_wg.png" width="400">
    <br>
    <i>SVM vs Softmax</i>
</div>
<br>

Fast R-CNN(FRCN)에서는 SVM보다 __Softmax__가 좋은 성능을 보여줬다.

또한, SVM를 사용하게되면 기존처럼 feature vector를 하드디스크에 저장해야되므로 stage-wise training 형태가 된다. 반면에 Softmax는 feature vector를 저장하지 않고 한번에 학습할 수 있다.

### 4.3 Region Proposals

아래 그림을 보면, region proposals의 개수를 증가시켜도 mAP는 비슷한 수치를 보여주고 있다. 따라서 성능을 향상시키기 위해 region proposal 수를 따로증가시킬 필요없이 기존 R-CNN에서 사용하던 __Selctive Search__로 충분하다는 점을 보여주고 있다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*66N_gcm4o7xAgeTb51HK0g.png" width="600">
    <br>
    <i>Different Proposal Approaches</i>
</div>
<br>

하지만, 외부소스를 이용한 region proposals 방법이 Fast R-CNN의 단점으로 뽑을 수 있다.

### 4.4  Truncated SVD for faster detection

testing time에 많은 시간이 걸리는 부분(bottleneck) 중 하나가 FC layer이다.

test time을 줄이기위해 논문에서 저자는 connection의 갯수를 줄이기위한 방법인 __Singular Vector Decomposition (SVD)__을 사용한다.

FC6 layer의 $25088×4096$ matrix에서 상위 1024개 singular values, FC7 layer의 $4096×4096$ matrix에서 상위 256 singular values만 사용한다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*Pcx4x1nUkmF8jiszfO8_nA.png" width="500">
    <br>
    <i>Large Reduction of Test Time for FC6 and FC7 Layers</i>
</div>
<br>

<a id='e'></a>
## 5. Comparison with State-of-the-art Results

### 5.1 VOC 2007

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*t4kHGY-VPUKexDiLC5ObzA.png" width="700">
    <br>
    <i>VOC 2007 Results</i>
</div>
<br>

* Fast R-CNN: 66.9% mAP
* Fast R-CNN with difficult examples removed during training (This is the setting of SPPNet): 68.1% mAP
* Fast R-CNN with external VOC 2012 trained: 70.0% mAP

### 5.2 VOC 2010

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*uH3kNBlBlLddtxB7zaVerQ.png" width="700">
    <br>
    <i>VOC 2010 Results</i>
</div>
<br>

VOC 2007과 비슷하게  Fast R-CNN은 68.8% mAP의 성능을 보여주었다.

### 5.3 VOC 2012

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*nEiW6axqzWZiXaVmokJF4w.png" width="700">
    <br>
    <i>VOC 2012 Results</i>
</div>
<br>

### 5.4 Training and Testing Time

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*boqOFsYjqYP2QJvLs2hA3w.png" width="500">
    <br>
    <i>Training and Testing Time</i>
</div>
<br>

표를 참고하면, Fast R-CNN은 VGG-16환경에서 R-CNN보다 9배 빠른 학습속도, 213배 빠른 테스트 속도를 보여줬다.

SPPNet과 비교해보면 3배 빠른 학습속도, 10배 빠른 테스트속도를 보여줬다.

## References

* [2015 ICCV] [Fast R-CNN]<br>
Fast R-CNN
* [2015 ICLR] [VGGNet]<br>
Very Deep Convolutional Networks for Large-Scale Image Recognition
* [2014 CVPR] [R-CNN]<br>
Rich feature hierarchies for accurate object detection and semantic segmentation
* [2014 ECCV] [SPPNet]<br>
Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition
* [2013 IJCV] [Selective Search]<br>
Selective Search for Object Recognition